# PROJECT-47. Исследование каналов привлечения

## ЛЕГЕНДА : 

### Руководитель клининговой компании обратился с просьбой выгрузить стратистику по конверсии в найм клинеров в направлении "Уборка".

## ЦЕЛЬ : 

### Определить коэффициент конверсии в найм клинеров.

## ЗАДАЧИ : 
### 1. Выгрузить данные.
### 2. Рассчитать коэффициент конверсии с помощью Pandas.

## ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ : 
### 1. Определить параметры запроса к Яндекс.Метрике.
### 2. Выгрузить их по API с помощью Jupyter Notebook.
### 3. Обработать данные в Pandas.
### 4. Посчитать коэффициент конверсии по каналам.

### ШАГ 1. Определить параметры запроса и выгрузить данные

- определение параметров запроса

In [1]:
# Импорт необходимых библиотек
import requests
import json
import pandas as pd
from pprint import pprint

# Добавление сервисного ключа - токена
token = 'y0_AQAAAABjzbrxAAhSRgAAAADLeXue6m1bxVOZSMiw5nR6v6aekQcMVz8'
headers = {'Authorization': 'OAuth ' + token}

# Введение необходимых параметров для выборки данных
params = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches', # 1 - метрика суммарного количества визитов, 2 - метрика количества выполнений целевого условия «Найм клинеров»
          'dimensions': 'ym:s:lastSignTrafficSource', # группировка модели «Последний значимый источник» по типу трафика
          'date1': '2024-05-01',  # начало отчетного периода - 1-е сентября 2024 
          'date2': '2024-09-30',  #конец отчетного периода - 30-е сентября 2024 
          'accuracy':'full',
          'limit':100000,
          'ids': 30177909}

- формирование запроса

In [2]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers) 
print (response.status_code) # 200 - запрос выполнен корректно

# Запись результатов в переменную в формате словаря (функция json())
metrika_data = response.json()

# Вывод первых 5 строк значения ключа data
pprint(metrika_data['data'][0:5])

200
[{'dimensions': [{'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'}],
  'metrics': [896943.0, 695.0]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [8891.0, 552.0]},
 {'dimensions': [{'icon_id': '1',
                  'icon_type': 'traffic-source',
                  'id': 'referral',
                  'name': 'Link traffic'}],
  'metrics': [512.0, 28.0]},
 {'dimensions': [{'icon_id': '3',
                  'icon_type': 'traffic-source',
                  'id': 'ad',
                  'name': 'Ad traffic'}],
  'metrics': [331.0, 119.0]},
 {'dimensions': [{'icon_id': '-1',
                  'icon_type': 'traffic-source',
                  'id': 'internal',
                  'name': 'Internal traffic'}],
  'metrics': [50.0, 3.0]}]


### ШАГ 2. Обработка данных в Pandas

- создание датафрейма, в который занесем все данные с запроса (ключ словаря - 'data')

In [3]:
metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head(10)) 

,dimensions,metrics
0,"[{'icon_id': '0', 'icon_type': 'traffic-source...","[896943.0, 695.0]"
1,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[8891.0, 552.0]"
2,"[{'icon_id': '1', 'icon_type': 'traffic-source...","[512.0, 28.0]"
3,"[{'icon_id': '3', 'icon_type': 'traffic-source...","[331.0, 119.0]"
4,"[{'icon_id': '-1', 'icon_type': 'traffic-sourc...","[50.0, 3.0]"
5,"[{'icon_id': '8', 'icon_type': 'traffic-source...","[28.0, 0.0]"
6,"[{'icon_id': '10', 'icon_type': 'traffic-sourc...","[14.0, 2.0]"
7,"[{'icon_id': '4', 'icon_type': 'traffic-source...","[1.0, 0.0]"


- избавление от вложенных словарей

In [4]:
# Применение функции getMetrikaDataInListOfDicts()
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:goal137595631reaches': 695.0,
  'ym:s:lastSignTrafficSource': 'Direct traffic',
  'ym:s:visits': 896943.0},
 {'ym:s:goal137595631reaches': 552.0,
  'ym:s:lastSignTrafficSource': 'Search engine traffic',
  'ym:s:visits': 8891.0},
 {'ym:s:goal137595631reaches': 28.0,
  'ym:s:lastSignTrafficSource': 'Link traffic',
  'ym:s:visits': 512.0},
 {'ym:s:goal137595631reaches': 119.0,
  'ym:s:lastSignTrafficSource': 'Ad traffic',
  'ym:s:visits': 331.0},
 {'ym:s:goal137595631reaches': 3.0,
  'ym:s:lastSignTrafficSource': 'Internal traffic',
  'ym:s:visits': 50.0}]


- создание нового датафрейма уже без вложенных словарей, а в виде списка

In [5]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
display(metrika_df.head(10))

,ym:s:lastSignTrafficSource,ym:s:visits,ym:s:goal137595631reaches
0,Direct traffic,896943.0,695.0
1,Search engine traffic,8891.0,552.0
2,Link traffic,512.0,28.0
3,Ad traffic,331.0,119.0
4,Internal traffic,50.0,3.0
5,Social network traffic,28.0,0.0
6,Messenger traffic,14.0,2.0
7,Cached page traffic,1.0,0.0


### ШАГ 3. Посчитать коэффициент конверсии по каналам

In [6]:
# Переименуем столбцы
metrika_df.columns=['Source','Visits','Conversions']

# Посчитаем конверсию модели атрибуции 'Last non-direct'
metrika_df['Last non-direct CR%'] = metrika_df['Conversions']/metrika_df['Visits']*100
display(metrika_df.head(10))

,Source,Visits,Conversions,Last non-direct CR%
0,Direct traffic,896943.0,695.0,0.077485
1,Search engine traffic,8891.0,552.0,6.208525
2,Link traffic,512.0,28.0,5.468750
3,Ad traffic,331.0,119.0,35.951662
4,Internal traffic,50.0,3.0,6.000000
5,Social network traffic,28.0,0.0,0.000000
6,Messenger traffic,14.0,2.0,14.285714
7,Cached page traffic,1.0,0.0,0.000000
